### Functional programming in Python
==> calling functions
* Regular functions created with def
* Anonymous functions created with lambda
* Instances of a class which define a \__call__ method
* Closures returned by functions
* Static methods of instances
* Generator functions

### Decoration and Delegation
Functional programming using classes
* Wrapping functions
* Delegation

### One more decorator example ...

In [ ]:
from random import randint

def with_retry(fun):
    num=3
    def retried(*args, **kwargs):
        exception = None
        for _ in range(num):
            try:
                return fun(*args, **kwargs)
            except Exception as e:
                print("Exception {} raised while calling {} with args: {}, kwargs: {}. Retrying".format(e, fun, args, kwargs))
                exception = e
        raise exception
    return retried

@with_retry
def blub():
    rnd = randint(0,5)
    print(rnd)
    if rnd < 5:
        raise Exception("Some exception occurred ...")

#blub = with_retry(blub)
    
blub()

## More complex decorator with arguments

In [ ]:
from random import randint

def with_retry(num=3):
    def wrap(fun):
        def retried(*args, **kwargs):
            exception = None
            print(num)
            for _ in range(num):
                try:
                    return fun(*args, **kwargs)
                except Exception as e:
                    print("Exception {} raised while calling {} with args: {}, kwargs: {}. Retrying".format(e, fun, args, kwargs))
                    exception = e
            raise exception
        return retried
    return wrap

@with_retry(100)
def blub():
    rnd = randint(0,5)
    print(rnd)
    if rnd < 5:
        raise Exception("Some exception occurred ...")
        
#blub = with_retry(100)(blub)    
blub()




## Now, implementation as a class.

In [ ]:
class decorating:
    def __init__(self, f):
        self.f = f
        print("Decorating ... {}".format(f))
        
    def __call__(self, *args, **kwargs):
        return self.f(*args, **kwargs)
    
@decorating
def foo(a, b):
    print(a, b)
    
foo(1, 2)

## Now, to something more complex ...

In [ ]:
from random import randint

class with_retry:
    
    def __init__(self, f):
        self.fun = f
        
    def __call__(self, *args, **kwargs):
        num=3
        exception = None
        for _ in range(num):
            try:
                return self.fun(*args, **kwargs)
            except Exception as e:
                print("Exception {} raised while calling {} with args: {}, kwargs: {}. Retrying".format(e, self.fun, args, kwargs))
                exception = e
        raise exception

@with_retry
def blub():
    rnd = randint(0,5)
    print(rnd)
    if rnd < 5:
        raise Exception("Some exception occurred ...")

#blub = with_retry(blub)
    
blub()

## Now, with arguments (looks a bit "cleaner" and simpler)

In [ ]:
class with_retry:
    
    def __init__(self, num):
        self.num = num
        
    def __call__(self, fun):
        def retried(*args, **kwargs):
            exception = None
            for _ in range(self.num):
                try:
                    return fun(*args, **kwargs)
                except Exception as e:
                    print("Exception {} raised while calling {} with args: {}, kwargs: {}. Retrying".format(e, fun, args, kwargs))
                    exception = e
            raise exception
        return retried

@with_retry(100)
def blub():
    rnd = randint(0,5)
    print(rnd)
    if rnd < 5:
        raise Exception("Some exception occurred ...")

blub = with_retry(100)(blub)
    
blub()

### Delegation
An object of a class A which "sends" all calls to an object of class B ...

Why is this useful?
* "Replace" inheritance by composition (if A is some container of B, then it "delegates" all calls to the contained object)
* Combine different objects into a larger object

In [ ]:
class Delegator:
    
    def __init__(self, delegate):
        self.delegate = delegate
        
    def __getattr__(self, name):
        attr = getattr(self.delegate, name)
        
        if not callable(attr):
            return attr
        
        def wrapper(*args, **kwargs):
            return attr(*args, **kwargs)
        return wrapper
    
    
class Delegate:
    def __init__(self):
        self.example = 123
        
    def doit(self, it):
        return "Doing {}".format(it)
    
delegator = Delegator(Delegate())

print(delegator.example)
delegator.doit("bla")
delegator.notimplemented("bar")


### More functional tools
* functools: reduce, many other useful functional programming tools!
* itertools: Iteration ...

In [ ]:
# partial function application ("currying")
from functools import partial

from operator import add

def add1(x):
    return add(1, x)

print(add1(2))

# could be written as

add1 = partial(add, 1)

add1(2)

## Reduce :-)

"So now reduce(). This is actually the one I've always hated most, because, apart from a few examples involving + or *, almost every time I see a reduce() call with a non-trivial function argument, I need to grab pen and paper to diagram what's actually being fed into that function before I understand what the reduce() is supposed to do. So in my mind, the applicability of reduce() is pretty much limited to associative operators, and in all other cases it's better to write out the accumulation loop explicitly."

Guido van Rossum, 2005

In [ ]:
from functools import reduce

reduce(lambda x, y: x * y, range(1,10))

In [ ]:
# Iterators

xs = [1, 2, 3]

it = iter(xs)

print(next(it))
print(next(it))
print(next(it))
# print(next(it))

# ==> next, get next element, saves state

In [ ]:
# Using generators

def lazy_integers(n=0):
    while True:
        yield n
        n += 1
        
xs = lazy_integers()

print([next(xs) for _ in range(10)])

# maintains state
[next(xs) for _ in range(10)]


In [ ]:
# Generator comprehensions

# computes nothing until next or for (lazy computation)
squares = (x**2 for x in lazy_integers())
doubles = (2*x for x in lazy_integers())

print(next(squares)) 
print(next(squares)) 
print(next(squares))

# do not do this!!!:
# bad_squares = [x**2 for x in lazy_integers()]


In [ ]:
from itertools import count

c = count(0, step=1)

print(next(c))
print(next(c))
print(next(c))

In [ ]:
from itertools import islice

# islice(seq, [start=0], stop, [step=1])
s = islice(range(0, 100000), 100, 200, 1)

print(next(s))
print(next(s))
print(next(s))

In [ ]:
from itertools import tee

# tee(it, [n=2])
# splits an iterator into two or more memoized copies
# huge efficiency gains if you have to iterate through expensive computations multiple times

print(list(tee(s, 2)[1]))


In [ ]:
from itertools import repeat

# repeat(elem, [n=forever])
# repeats elem n times (or forever if no n)
repeat(5, 1000)

In [ ]:
from itertools import cycle

# cycle(p)
# repeats the elements of p over and over and over again forever

c = cycle([1, 2, 3])
for i in range(1, 10):
    print(next(c))


In [ ]:
from itertools import chain
# chain(p, q, …)
# iterates first through the elements of p, then the elements of q, and so on

c = chain([1, 2, 3], [7, 8, 9])
for i in range(1, 7):
    print(next(c))


In [ ]:
from itertools import accumulate

# accumulate(p, [func=add])
# returns the sequence a, where
# a[0] = p[0]
# a[1] = func(a[0], p[1])
# a[2] = func(a[1], p[2])

list(accumulate([1, 2, 3]))


In [ ]:
# some more itertools

# force the first n values of a sequence
def take(n, it):
    return [x for x in islice(it, n)]

# new sequence with all but the first n values of a sequence
def drop(n, it):
    return islice(it, n, None)

# force the first value of a sequence
head = next

# new sequence with all but the first value of a sequence
tail = partial(drop, 1)



In [ ]:
# Creating - iterate -
# iterate(f, x)
# should be the sequence x, f(x), f(f(x)), ...

def iterate(f, x):
    return accumulate(repeat(x), lambda fx, _: f(fx))



In [ ]:
def lazy_integers():
    return iterate(add1, 0)

take(10, lazy_integers())

### Now Fibonacci numbers

In [ ]:
def fib(n):
    if n == 0:
        return 1
    
    if n == 1:
        return 1
    
    return fib(n-1) + fib(n-2)

print([fib(i) for i in range(10)])

%time fib(30)
# CPU times: user 293 ms, sys: 2.44 ms, total: 295 ms
# Wall time: 294 ms

In [ ]:
# also not efficient, really ...

def fibs():
    yield 1
    yield 1
    yield from map(add, fibs(), tail(fibs()))
    
take(10, fibs())

%time take(30, fibs())
#CPU times: user 7.38 s, sys: 439 ms, total: 7.82 s
# Wall time: 7.83 s


In [ ]:
# improved again

def fibs():
    yield 1
    yield 1
    fibs1, fibs2 = tee(fibs())
    yield from map(add, fibs1, tail(fibs2))

%time take(30, fibs())
# CPU times: user 131 µs, sys: 39 µs, total: 170 µs
# Wall time: 175 µs


In [ ]:
def next_fib(pair):
    x, y = pair
    return (y, x + y)

def fibs():
    return (y for x, y in iterate(next_fib, (0, 1)))

%time take(30, fibs())
# CPU times: user 22 µs, sys: 0 ns, total: 22 µs
# Wall time: 26 µs
